In [1]:
import pandas as pd
import re
from helper.onemap import OneMapQuery, convert_second_to_time_with_s
from tqdm import tqdm
import time

In [2]:
num_vehicles = int(input("Enter number of vehicles: "))
working_hours = float(input("Enter working hours per vehicle: "))
working_seconds = int(working_hours * 3600)  # Convert hours to seconds

print(f"\nConfiguration:")
print(f"Number of vehicles: {num_vehicles}")
print(f"Working hours: {working_hours} hours ({working_seconds} seconds)")


Configuration:
Number of vehicles: 5
Working hours: 1.2 hours (4320 seconds)


In [3]:
# Read the Excel file
df = pd.read_excel('store/data/travelling_salesman.xlsx')
df

,job_id,address
0,1,11 WOODLANDS INDUSTRIAL PARK E1 SINGAPORE 757734
1,2,58 JALAN TANAH PUTEH SINGAPORE 457360
2,3,97 YISHUN STREET 81 ORCHID PARK CONDOMINIUM SI...
3,4,11 CHIN BEE DRIVE SINGAPORE 619862
4,5,151 PUNGGOL CENTRAL NEIGHBOURHOOD POLICE CENTR...
5,6,1 ROBIN ROAD ONE ROBIN SINGAPORE 258176
6,7,23 KIM YAM ROAD WATERFORD RESIDENCE SINGAPORE ...
7,8,501 ORCHARD ROAD WHEELOCK PLACE SINGAPORE 238880
8,9,303 HENDERSON ROAD SAINT ANDREW'S NURSING HOME...
9,10,571 UPPER SERANGOON ROAD THE YARDLEY SINGAPORE...


In [4]:
om = OneMapQuery()
depot_latlong = (1.31158791334051, 103.863375124429)  # CT Hub 2
all_locations = [depot_latlong]  # Start with depot

In [5]:
print("Processing locations...")
location_index_map = {}  # Map postal code to location index
for idx, row in tqdm(df.iterrows(), total=len(df)):
    address = row['address']
    # Extract postal code using regex
    postal_match = re.search(r'SINGAPORE (\d{6})', address)
    if postal_match:
        postal = postal_match.group(1)
        latlong = om.get_postal_latlong(postal)
        if latlong:
            # Check if this location is already in our list
            if postal not in location_index_map:
                all_locations.append(latlong)
                location_index_map[postal] = len(all_locations) - 1  # Map postal code to index
            # Add the location index to the dataframe
            df.loc[idx, 'location_index'] = location_index_map[postal]
    time.sleep(0.5)  # Add delay to avoid hitting rate limits

print(f"\nUnique locations processed: {len(all_locations)-1}")  # -1 to exclude depot

Processing locations...


100%|██████████| 30/30 [00:27<00:00,  1.09it/s]


Unique locations processed: 30


In [6]:
print("\nCalculating route matrices...")
duration_matrix, distance_matrix = om.get_route_matrices(all_locations)


Calculating route matrices...


In [7]:
from vroom import Vehicle, Job, Input, TimeWindow

# Create VROOM input
vroom_input = Input()

# Add vehicles with time windows
for i in range(num_vehicles):
    vehicle = Vehicle(
        id=i+1,  # Vehicle ID (1-based)
        start=0,  # Depot location index
        end=0,    # Return to depot
        time_window=TimeWindow(
            start=0,              # Start time (0 seconds)
            end=working_seconds   # End time (user-specified)
        )
    )
    vroom_input.add_vehicle(vehicle)

In [8]:
# Create jobs from the dataframe
for _, row in df.iterrows():
    job = Job(
        id=row['job_id'],  # Use job_id from dataframe
        location=int(row['location_index'])  # Use location_index we created, convert to int
    )
    vroom_input.add_job(job)

In [9]:
vroom_input.set_durations_matrix(
    profile="car",
    matrix_input=duration_matrix
)
vroom_input.set_distances_matrix(
    profile="car",
    matrix_input=distance_matrix
)

In [10]:
solution = vroom_input.solve(exploration_level=5, nb_threads=4)

# Convert solution routes to a dataframe for easier analysis
route_df = solution.routes.copy()

print("\nOptimization Results:")
print(f"Total duration: {convert_second_to_time_with_s(solution.summary.duration)}")
print(f"Total distance: {solution.summary.distance} meters")


Optimization Results:
Total duration: 05:43:04
Total distance: 186530 meters


In [11]:
import folium

# Create a map centered on Singapore
m = folium.Map(location=[1.352083, 103.819839], zoom_start=12, tiles="cartodbpositron")

# Add depot marker with special styling
folium.CircleMarker(
    location=depot_latlong,
    popup='Depot (CT Hub 2)',
    tooltip='Depot',
    color='red',
    fill=True,
    fillColor='red',
    radius=10
).add_to(m)

In [12]:
# Group routes by vehicle
for vehicle_id in range(1, num_vehicles + 1):
    vehicle_route = route_df[route_df['vehicle_id'] == vehicle_id]
    route_sequence = []
    
    # Get sequence of locations for this vehicle
    for step in vehicle_route.itertuples():
        route_sequence.append(all_locations[step.location_index])
    
    # Plot routes between consecutive points
    for i in range(len(route_sequence)-1):
        start = route_sequence[i]
        end = route_sequence[i+1]
        # Plot route with sequence numbers, using vehicle_id as color index
        om.plot_routes(start, end, m, vehicle_id, (i, i+1))

In [13]:
m

In [14]:
m.save('optimized_route_with_time.html')

# Get all job IDs from the solution (excluding start/end rows)
completed_jobs = route_df[route_df['type'] == 'job']['id'].tolist()

# Get all job IDs from our original dataframe
all_jobs = df['job_id'].tolist()

# Check for missing jobs
missing_jobs = set(all_jobs) - set(completed_jobs)

if len(missing_jobs) == 0:
    print("All jobs were fulfilled!")
else:
    print("Warning: Some jobs were not fulfilled!")
    print("Missing jobs:", sorted(list(missing_jobs)))
    
print("\nMap has been saved as 'optimized_route_with_time.html'")
print(f"Total vehicles used: {len(route_df['vehicle_id'].unique())}")
print(f"Total stops: {len(completed_jobs)}")

Missing jobs: [1, 17, 29]

Map has been saved as 'optimized_route_with_time.html'
Total vehicles used: 5
Total stops: 27


In [15]:
print("\nRoute Details per Vehicle:")
for vehicle_id in range(1, num_vehicles + 1):
    vehicle_route = route_df[route_df['vehicle_id'] == vehicle_id]
    if not vehicle_route.empty:
        print(f"\nVehicle {vehicle_id}:")
        print(f"Start time: {convert_second_to_time_with_s(vehicle_route.iloc[0]['arrival'])}")
        print(f"End time: {convert_second_to_time_with_s(vehicle_route.iloc[-1]['arrival'])}")
        print(f"Total duration: {convert_second_to_time_with_s(vehicle_route.iloc[-1]['duration'])}")
        print(f"Total distance: {vehicle_route.iloc[-1]['distance']} meters")
        print(f"Number of stops: {len(vehicle_route[vehicle_route['type'] == 'job'])}")


Route Details per Vehicle:

Vehicle 1:
Start time: 00:00:00
End time: 01:08:31
Total duration: 01:08:31
Total distance: 30857 meters
Number of stops: 10

Vehicle 2:
Start time: 00:00:00
End time: 01:11:43
Total duration: 01:11:43
Total distance: 40484 meters
Number of stops: 5

Vehicle 3:
Start time: 00:00:00
End time: 01:11:39
Total duration: 01:11:39
Total distance: 40318 meters
Number of stops: 3

Vehicle 4:
Start time: 00:00:00
End time: 01:10:59
Total duration: 01:10:59
Total distance: 46914 meters
Number of stops: 4

Vehicle 5:
Start time: 00:00:00
End time: 01:00:12
Total duration: 01:00:12
Total distance: 27957 meters
Number of stops: 5
